# [Product Recommendations for Online Retail Store](https://medium.com/@peggy1502/product-recommendations-for-online-retail-store-1d565e1607b7)
### Build and Train a Personalized Recommender Engine with Amazon Sagemaker Factorization Machines

**This is `Notebook Part 2`**

**Click [here](fm_v3_part1.ipynb) for `Notebook Part 1`**

In [14]:
import numpy as np 
import pandas as pd 
import time

import boto3

from scipy.sparse import csr_matrix, hstack, save_npz, load_npz
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [15]:
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("boto3 version:", boto3.__version__)

numpy version: 1.19.5
pandas version: 1.1.5
boto3 version: 1.19.3


In [19]:
import sagemaker
import sagemaker.amazon.common as smac

sagemaker.__version__

'2.63.2'

# Reading npz files

In [16]:
# load array and sparse matrices.

X_train = load_npz("X_train.npz")
X_test = load_npz("X_test.npz")

y_train = np.load("y_train.npz")
y_test = np.load("y_test.npz")
y_train = y_train.f.arr_0
y_test = y_test.f.arr_0

# Example of sparse matrix for X_test
# pd.DataFrame(X_test.todense())

In [17]:
feature_dim = 0

# Read the saved feature dimension.
with open("feature_dim.txt", "r") as f:
    feature_dim = int(f.read())
    
feature_dim

211030

# Creating Sparse RecordIO File

https://docs.aws.amazon.com/sagemaker/latest/dg/fact-machines.html

For **training**, the **Factorization Machines** algorithm currently supports only the `recordIO-protobuf` format with `Float32` tensors.

For **inference**, the **Factorization Machines** algorithm supports the `application/json` and `x-recordio-protobuf` formats.

In [20]:
# Function to create sparse RecordIO file.

def write_sparse_recordio_file (filename, X, y=None):
    with open(filename, 'wb') as f:
        smac.write_spmatrix_to_sparse_tensor (f, X, y)

In [21]:
# Function to upload file to S3.
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-uploading-files.html
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.upload_fileobj

def upload_to_s3(filename, bucket, prefix, key):
    with open(filename,'rb') as f: # Read in binary mode
        boto3.Session().resource('s3').Bucket(bucket).Object(f"{prefix}/{key}").upload_fileobj(f)
        return f"s3://{bucket}/{prefix}/{key}"

In [22]:
# Creating the train and test RecordIO files.

write_sparse_recordio_file("fm_train.recordio", X_train, y_train)
write_sparse_recordio_file("fm_test.recordio", X_test, y_test)

In [23]:
# Uploading the train and test RecordIO files to S3.

sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

prefix = "fm"
train_key = "fm_train.recordio"
test_key = "fm_test.recordio"
output_location = f"s3://{bucket}/{prefix}/output"

train_file_location = upload_to_s3("fm_train.recordio", bucket, prefix, train_key)
test_file_location = upload_to_s3("fm_test.recordio", bucket, prefix, test_key)

print("SageMaker version:", sagemaker.__version__)
print("Region:", region)
print("Bucket:", bucket)
print("train file location:", train_file_location)
print("test file location:", test_file_location)
print("model output location:", output_location)

SageMaker version: 2.63.2
Region: us-east-2
Bucket: sagemaker-us-east-2-802795124455
train file location: s3://sagemaker-us-east-2-802795124455/fm/fm_train.recordio
test file location: s3://sagemaker-us-east-2-802795124455/fm/fm_test.recordio
model output location: s3://sagemaker-us-east-2-802795124455/fm/output


# Training Job & Hyperparameters

In [24]:
job_name = 'fm-ecommerce-v3-' + 
            time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
job_name

'fm-ecommerce-v3-2021-10-30-02-20-41'

In [25]:
# https://github.com/aws-samples/amazon-sagemaker-managed-spot-training/blob/main/xgboost_built_in_managed_spot_training_checkpointing/xgboost_built_in_managed_spot_training_checkpointing.ipynb
    
use_spot_instances = False
max_run = 3600                                   # set to 60 mins
max_wait = 3600 if use_spot_instances else None  # set to 60 mins (must be equal or greater than max_run)
   
checkpoint_s3_uri = (f"s3://{bucket}/{prefix}/checkpoints/{job_name}" if use_spot_instances
                     else None)
    
print(f"Checkpoint uri: {checkpoint_s3_uri}")

Checkpoint uri: None


In [26]:
role = sagemaker.get_execution_role()
role

'arn:aws:iam::802795124455:role/service-role/AmazonSageMaker-ExecutionRole-20211026T153321'

In [27]:
container = sagemaker.image_uris.retrieve("factorization-machines", region=region)
container

'404615174143.dkr.ecr.us-east-2.amazonaws.com/factorization-machines:1'

In [ ]:
estimator = sagemaker.estimator.Estimator(    
    container,
    role,
    instance_count = 1,
    instance_type = "ml.m4.xlarge",   # Or "ml.c5.xlarge",
    output_path = output_location,
    sagemaker_session = sess,
    base_job_name = job_name,
    use_spot_instances = use_spot_instances,
    max_run = max_run,
    max_wait = max_wait,
    checkpoint_s3_uri = checkpoint_s3_uri
)

In [28]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/fact-machines-hyperparameters.html

estimator.set_hyperparameters(
    feature_dim = feature_dim,
    num_factors = 64,  
    predictor_type = "regressor",
    epochs = 88,      
    mini_batch_size = 1000,  
)

In [29]:
estimator.hyperparameters()

{'feature_dim': 211030,
 'num_factors': 64,
 'predictor_type': 'regressor',
 'epochs': 88,
 'mini_batch_size': 1000}

# Train Model

In [30]:
estimator.fit({'train':train_file_location, 
               'test':test_file_location})

2021-10-30 02:23:13 Starting - Starting the training job...
2021-10-30 02:23:37 Starting - Launching requested ML instancesProfilerReport-1635560593: InProgress
...
2021-10-30 02:24:08 Starting - Preparing the instances for training.........
2021-10-30 02:25:38 Downloading - Downloading input data...
2021-10-30 02:25:58 Training - Downloading the training image...
2021-10-30 02:26:38 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python3.7/site-packages/jsonref.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping, Sequence
/opt/amazon/lib/python3.7/site-packages/algorithm/network_builder.py:87: DeprecationWarning: invalid escape sequence \s
  """
/opt/amazon/lib/python3.7/site-packages/alg

[2021-10-30 02:26:52.805] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 4, "duration": 5069, "num_examples": 322, "num_bytes": 33805260}
[10/30/2021 02:26:52 INFO 140714915637056] #quality_metric: host=algo-1, epoch=1, train rmse <loss>=8.834326277883381
[10/30/2021 02:26:52 INFO 140714915637056] #quality_metric: host=algo-1, epoch=1, train mse <loss>=78.04532078410084
[10/30/2021 02:26:52 INFO 140714915637056] #quality_metric: host=algo-1, epoch=1, train absolute_loss <loss>=1.189023306141729
#metrics {"StartTime": 1635560807.7316608, "EndTime": 1635560812.805964, "Dimensions": {"Algorithm": "factorization-machines", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 5072.132587432861, "count": 1, "min": 5072.132587432861, "max": 5072.132587432861}}}
[10/30/2021 02:26:52 INFO 140714915637056] #progress_metric: host=algo-1, completed 2.272727272727273 % of epochs
#metrics {"StartTime": 1635560807.733796, "EndTime": 1635

[2021-10-30 02:27:23.650] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 16, "duration": 4854, "num_examples": 322, "num_bytes": 33805260}
[10/30/2021 02:27:23 INFO 140714915637056] #quality_metric: host=algo-1, epoch=7, train rmse <loss>=9.9512748800453
[10/30/2021 02:27:23 INFO 140714915637056] #quality_metric: host=algo-1, epoch=7, train mse <loss>=99.0278717382206
[10/30/2021 02:27:23 INFO 140714915637056] #quality_metric: host=algo-1, epoch=7, train absolute_loss <loss>=1.9195738822984398
#metrics {"StartTime": 1635560838.7917287, "EndTime": 1635560843.6509855, "Dimensions": {"Algorithm": "factorization-machines", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 4857.171058654785, "count": 1, "min": 4857.171058654785, "max": 4857.171058654785}}}
[10/30/2021 02:27:23 INFO 140714915637056] #progress_metric: host=algo-1, completed 9.090909090909092 % of epochs
#metrics {"StartTime": 1635560838.7937844, "EndTime": 163

[2021-10-30 02:27:43.180] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 24, "duration": 5005, "num_examples": 322, "num_bytes": 33805260}
[10/30/2021 02:27:43 INFO 140714915637056] #quality_metric: host=algo-1, epoch=11, train rmse <loss>=10.130703596412047
[10/30/2021 02:27:43 INFO 140714915637056] #quality_metric: host=algo-1, epoch=11, train mse <loss>=102.63115535835598
[10/30/2021 02:27:43 INFO 140714915637056] #quality_metric: host=algo-1, epoch=11, train absolute_loss <loss>=2.1028308903474984
#metrics {"StartTime": 1635560858.1707847, "EndTime": 1635560863.1811872, "Dimensions": {"Algorithm": "factorization-machines", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 5008.260011672974, "count": 1, "min": 5008.260011672974, "max": 5008.260011672974}}}
[10/30/2021 02:27:43 INFO 140714915637056] #progress_metric: host=algo-1, completed 13.636363636363637 % of epochs
#metrics {"StartTime": 1635560858.172897, "EndTi

[2021-10-30 02:28:02.856] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 32, "duration": 5050, "num_examples": 322, "num_bytes": 33805260}
[10/30/2021 02:28:02 INFO 140714915637056] #quality_metric: host=algo-1, epoch=15, train rmse <loss>=11.265372992721643
[10/30/2021 02:28:02 INFO 140714915637056] #quality_metric: host=algo-1, epoch=15, train mse <loss>=126.90862866514217
[10/30/2021 02:28:02 INFO 140714915637056] #quality_metric: host=algo-1, epoch=15, train absolute_loss <loss>=2.5379479152134485
#metrics {"StartTime": 1635560877.8017163, "EndTime": 1635560882.8568435, "Dimensions": {"Algorithm": "factorization-machines", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 5053.015232086182, "count": 1, "min": 5053.015232086182, "max": 5053.015232086182}}}
[10/30/2021 02:28:02 INFO 140714915637056] #progress_metric: host=algo-1, completed 18.181818181818183 % of epochs
#metrics {"StartTime": 1635560877.8037994, "EndT

...

[2021-10-30 02:32:21.944] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 136, "duration": 4886, "num_examples": 322, "num_bytes": 33805260}
[10/30/2021 02:32:21 INFO 140714915637056] #quality_metric: host=algo-1, epoch=67, train rmse <loss>=8.797594254153822
[10/30/2021 02:32:21 INFO 140714915637056] #quality_metric: host=algo-1, epoch=67, train mse <loss>=77.39766466072035
[10/30/2021 02:32:21 INFO 140714915637056] #quality_metric: host=algo-1, epoch=67, train absolute_loss <loss>=1.1828566549549933
#metrics {"StartTime": 1635561137.0535917, "EndTime": 1635561141.9458494, "Dimensions": {"Algorithm": "factorization-machines", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 4889.866352081299, "count": 1, "min": 4889.866352081299, "max": 4889.866352081299}}}
[10/30/2021 02:32:21 INFO 140714915637056] #progress_metric: host=algo-1, completed 77.27272727272727 % of epochs
#metrics {"StartTime": 1635561137.0559494, "EndTim

[2021-10-30 02:32:46.337] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 146, "duration": 4977, "num_examples": 322, "num_bytes": 33805260}
[10/30/2021 02:32:46 INFO 140714915637056] #quality_metric: host=algo-1, epoch=72, train rmse <loss>=9.219008800151766
[10/30/2021 02:32:46 INFO 140714915637056] #quality_metric: host=algo-1, epoch=72, train mse <loss>=84.9901232572757
[10/30/2021 02:32:46 INFO 140714915637056] #quality_metric: host=algo-1, epoch=72, train absolute_loss <loss>=1.3323067096212635
#metrics {"StartTime": 1635561161.3555932, "EndTime": 1635561166.338951, "Dimensions": {"Algorithm": "factorization-machines", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 4980.929613113403, "count": 1, "min": 4980.929613113403, "max": 4980.929613113403}}}
[10/30/2021 02:32:46 INFO 140714915637056] #progress_metric: host=algo-1, completed 82.95454545454545 % of epochs
#metrics {"StartTime": 1635561161.3579628, "EndTime"

[2021-10-30 02:33:11.641] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 156, "duration": 4910, "num_examples": 322, "num_bytes": 33805260}
[10/30/2021 02:33:11 INFO 140714915637056] #quality_metric: host=algo-1, epoch=77, train rmse <loss>=8.841362580040256
[10/30/2021 02:33:11 INFO 140714915637056] #quality_metric: host=algo-1, epoch=77, train mse <loss>=78.1696922717361
[10/30/2021 02:33:11 INFO 140714915637056] #quality_metric: host=algo-1, epoch=77, train absolute_loss <loss>=1.1683158252787145
#metrics {"StartTime": 1635561186.72632, "EndTime": 1635561191.6418748, "Dimensions": {"Algorithm": "factorization-machines", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 4913.248062133789, "count": 1, "min": 4913.248062133789, "max": 4913.248062133789}}}
[10/30/2021 02:33:11 INFO 140714915637056] #progress_metric: host=algo-1, completed 88.63636363636364 % of epochs
#metrics {"StartTime": 1635561186.728592, "EndTime": 

[2021-10-30 02:33:31.218] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 164, "duration": 4820, "num_examples": 322, "num_bytes": 33805260}
[10/30/2021 02:33:31 INFO 140714915637056] #quality_metric: host=algo-1, epoch=81, train rmse <loss>=8.736777408542876
[10/30/2021 02:33:31 INFO 140714915637056] #quality_metric: host=algo-1, epoch=81, train mse <loss>=76.33127948642517
[10/30/2021 02:33:31 INFO 140714915637056] #quality_metric: host=algo-1, epoch=81, train absolute_loss <loss>=1.0825448195179057
#metrics {"StartTime": 1635561206.3936946, "EndTime": 1635561211.2189252, "Dimensions": {"Algorithm": "factorization-machines", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 4822.966575622559, "count": 1, "min": 4822.966575622559, "max": 4822.966575622559}}}
[10/30/2021 02:33:31 INFO 140714915637056] #progress_metric: host=algo-1, completed 93.18181818181819 % of epochs
#metrics {"StartTime": 1635561206.395926, "EndTime

[2021-10-30 02:33:55.679] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 174, "duration": 4782, "num_examples": 322, "num_bytes": 33805260}
[10/30/2021 02:33:55 INFO 140714915637056] #quality_metric: host=algo-1, epoch=86, train rmse <loss>=8.883343536186285
[10/30/2021 02:33:55 INFO 140714915637056] #quality_metric: host=algo-1, epoch=86, train mse <loss>=78.91379238190265
[10/30/2021 02:33:55 INFO 140714915637056] #quality_metric: host=algo-1, epoch=86, train absolute_loss <loss>=1.2241861991171512
#metrics {"StartTime": 1635561230.8933496, "EndTime": 1635561235.680538, "Dimensions": {"Algorithm": "factorization-machines", "Host": "algo-1", "Operation": "training"}, "Metrics": {"update.time": {"sum": 4784.824848175049, "count": 1, "min": 4784.824848175049, "max": 4784.824848175049}}}
[10/30/2021 02:33:55 INFO 140714915637056] #progress_metric: host=algo-1, completed 98.86363636363636 % of epochs
#metrics {"StartTime": 1635561230.8956828, "EndTime

In [ ]:
# # training_job_info = sagemaker_boto_client.describe_training_job(TrainingJobName=job_name)
# boto3_client = boto3.client("sagemaker")
# training_job_info = boto3_client.describe_training_job(TrainingJobName=job_name)
# training_job_info

# Deploy Model

In [39]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer
import json

class fm_json_serializer(JSONSerializer):
    def serialize(self, data):
        js = {"instances": []}
        for row in data:
            js["instances"].append({"features": row.tolist()})
        return json.dumps(js)

In [40]:
predictor = estimator.deploy(initial_instance_count = 1,
                             instance_type = "ml.m5.xlarge",
                             endpoint_name = job_name,
                             serializer = fm_json_serializer(),
                             deserializer = JSONDeserializer(),
                            )

-----!

# Model Inference

### Top Spender

In [41]:
# Find customer who spent the most money

df = pd.read_csv("fm_preprocessed.zip")
df["category_name_1"].fillna("", inplace=True)
df["invoice_amount"] = df["qty_ordered"] * df["price"]

top_spender = (df.groupby("Customer ID").sum()["invoice_amount"].sort_values(ascending=False).index[0])
print("Customer ID of top spender:", top_spender)

Customer ID of top spender: 5032.0


In [42]:
# Transaction history of top spender.

df[df["Customer ID"]==5032].head(60)

,sku,category_name_1,sku_and_cat,Customer ID,price,qty_ordered,invoice_amount
2649,AKL_A131135338-FW-18-Orange,Women's Fashion,AKL A131135338 FW 18 Orange Women's Fashion,5032.0,2600.0,1.0,2600.0
3528,AKL_A131138782-SS-117-17,Women's Fashion,AKL A131138782 SS 117 17 Women's Fashion,5032.0,2400.0,1.0,2400.0
3541,AKL_A131138785-SS-119-17,Women's Fashion,AKL A131138785 SS 119 17 Women's Fashion,5032.0,2400.0,1.0,2400.0
3548,AKL_A131138786-SS-119-17,Women's Fashion,AKL A131138786 SS 119 17 Women's Fashion,5032.0,2400.0,1.0,2400.0
3670,AKL_A131138806-SS-130-17,Women's Fashion,AKL A131138806 SS 130 17 Women's Fashion,5032.0,2400.0,1.0,2400.0
3676,AKL_A131138807-SS-131-17,Women's Fashion,AKL A131138807 SS 131 17 Women's Fashion,5032.0,2400.0,1.0,2400.0
8916,APPDAW59FEF7B84F84B,Appliances,APPDAW59FEF7B84F84B Appliances,5032.0,9894.0,1.0,9894.0
37338,Anchor_439-L,Men's Fashion,Anchor 439 L Men's Fashion,5032.0,499.0,1.0,499.0
37536,Anchor_60-L,Men's Fashion,Anchor 60 L Men's Fashion,5032.0,549.0,1.0,549.0
81826,Delsey_207382001,Home & Living,Delsey 207382001 Home & Living,5032.0,10656.0,1.0,10656.0


### Top Trending Products

In [95]:
trending = (
        df.groupby(["sku", "category_name_1", "price"]).nunique()["Customer ID"]
        .sort_values(ascending=False)
        .reset_index()
    )

# https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.nunique.html
trending = trending.rename(columns={'Customer ID': 'unique_customers'})
trending

,sku,category_name_1,price,unique_customers
0,MATSAM59DB75ADB2F80,Mobiles & Tablets,13698.0,652
1,emart_00-7,Home & Living,699.0,649
2,emart_00-1,Others,649.0,601
3,unilever_Deal-6,Superstore,370.0,590
4,Al Muhafiz Sohan Halwa Almond,Soghaat,388.0,580
...,...,...,...,...
101904,MEFSWA5A003FA7F2C7F,Men's Fashion,5940.0,1
101905,MEFSWA5A003FA56201C,Men's Fashion,6600.0,1
101906,MEFSWA5A003FA41FB65,Men's Fashion,9180.0,1
101907,MEFSWA5A003FA049F8E,Men's Fashion,9180.0,1


In [96]:
top_5_products = trending["sku"].iloc[:5].values.tolist()
top_5_prices = trending["price"].iloc[:5].values.tolist()
top_5_cat_name = trending["category_name_1"].iloc[:5].values.tolist()

In [97]:
data = {
        "sku": top_5_products,
        "category_name_1": top_5_cat_name,
        "Customer ID": top_spender,        
        "price": top_5_prices,
    }
df_top_5 = pd.DataFrame(data)
df_top_5["sku_and_cat"] = df_top_5["sku"] + " " + df_top_5["category_name_1"]
df_top_5

,sku,category_name_1,Customer ID,price,sku_and_cat
0,MATSAM59DB75ADB2F80,Mobiles & Tablets,5032.0,13698.0,MATSAM59DB75ADB2F80 Mobiles & Tablets
1,emart_00-7,Home & Living,5032.0,699.0,emart_00-7 Home & Living
2,emart_00-1,Others,5032.0,649.0,emart_00-1 Others
3,unilever_Deal-6,Superstore,5032.0,370.0,unilever_Deal-6 Superstore
4,Al Muhafiz Sohan Halwa Almond,Soghaat,5032.0,388.0,Al Muhafiz Sohan Halwa Almond Soghaat


In [98]:
ohe = OneHotEncoder(handle_unknown = "ignore")
ohe_cols = ["sku", "category_name_1", "Customer ID"]
ohe.fit(df[ohe_cols])
ohe_features = ohe.transform(df_top_5[ohe_cols])
ohe_features

<5x200137 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [99]:
df["sku_and_cat"] = df["sku_and_cat"].str.replace("-", " ")
df["sku_and_cat"] = df["sku_and_cat"].str.replace("_", " ")

vectorizer = TfidfVectorizer(min_df=2)  # Ignore terms that appear in less than 2 documents.
vectorizer.fit(df["sku_and_cat"].unique())
tfidf_features = vectorizer.transform(df_top_5["sku_and_cat"])
tfidf_features

<5x10892 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [100]:
row = range(len(df_top_5))
col = [0] * len(df_top_5)   # This is a list of zeros [0,0,0,....]
price = csr_matrix((df_top_5["price"].values, (row, col)), dtype="float32")
price

<5x1 sparse matrix of type '<class 'numpy.float32'>'
	with 5 stored elements in Compressed Sparse Row format>

In [101]:
X_top_5 = hstack([ohe_features, tfidf_features, price], format="csr", dtype="float32")
X_top_5

<5x211030 sparse matrix of type '<class 'numpy.float32'>'
	with 32 stored elements in Compressed Sparse Row format>

In [102]:
X_top_5.toarray()
# pd.DataFrame(X_top_5.toarray())

array([[    0.,     0.,     0., ...,     0.,     0., 13698.],
       [    0.,     0.,     0., ...,     0.,     0.,   699.],
       [    0.,     0.,     0., ...,     0.,     0.,   649.],
       [    0.,     0.,     0., ...,     0.,     0.,   370.],
       [    0.,     0.,     0., ...,     0.,     0.,   388.]],
      dtype=float32)

### Get recommendation for top spender from the top 5 trending products

In [103]:
result = predictor.predict(X_top_5.toarray())
result

{'predictions': [{'score': 33.07421875},
  {'score': 5.309963226318359},
  {'score': 6.309589385986328},
  {'score': 4.952110290527344},
  {'score': 4.305643081665039}]}

In [104]:
predictions = [i["score"] for i in result["predictions"]]
predictions

[33.07421875,
 5.309963226318359,
 6.309589385986328,
 4.952110290527344,
 4.305643081665039]

In [105]:
# argsort: smaller values are in front, bigger values are behind.

index_array = np.array(predictions).argsort()
index_array

array([4, 3, 1, 2, 0])

In [106]:
skus = ohe.inverse_transform(ohe_features)[:, 0]
skus

array(['MATSAM59DB75ADB2F80', 'emart_00-7', 'emart_00-1',
       'unilever_Deal-6', 'Al Muhafiz Sohan Halwa Almond'], dtype=object)

### Top 3 recommendations for top spender

In [107]:
# Top 3 recommendations means take the biggest values from behind 
# (i.e. index 0 followed by index 2 and 1).

top_3_recommended = np.take_along_axis(skus, index_array, axis=0)[: -3 - 1 : -1]
top_3_recommended

array(['MATSAM59DB75ADB2F80', 'emart_00-1', 'emart_00-7'], dtype=object)